<h4>Global CO<sub>2</sub> Emissions Dataset from Maven Analytics</h4>

<p>Objectives:</p>
<ul>
    <li>What is the trend of CO<sub>2</sub> emissions in the world over time?
    Can it be related to the demographics?</li>
    <li>Which country contributes both the least and the most emissions?</li>
    <li>What are the major sources of these emissions? </li>
    <li>Is temperature related to the amount of CO<sub>2</sub> emissions?</li>
</ul>

In [504]:
#Libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [505]:
df = pd.read_csv("data/co2_data.csv")
df.shape

(50598, 79)

In [506]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50598 entries, 0 to 50597
Data columns (total 79 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   country                                    50598 non-null  object 
 1   year                                       50598 non-null  int64  
 2   iso_code                                   42142 non-null  object 
 3   population                                 40008 non-null  float64
 4   gdp                                        14564 non-null  float64
 5   cement_co2                                 24974 non-null  float64
 6   cement_co2_per_capita                      22714 non-null  float64
 7   co2                                        31349 non-null  float64
 8   co2_growth_abs                             29010 non-null  float64
 9   co2_growth_prct                            25032 non-null  float64
 10  co2_including_luc     

<p>Dividing the data columns based on the objectives.</p>

In [507]:
#Columns containing demographics and CO2 emissions
df_demographics = df[['country','iso_code','year', 'population', 'gdp','co2',
'land_use_change_co2']]
df_demographics = df_demographics.dropna(subset=['iso_code'])

# Columns containing the sources of emissions 
df_sources = df[['cement_co2','coal_co2','flaring_co2','gas_co2','land_use_change_co2',\
    'consumption_co2','oil_co2','other_industry_co2']]

# Columns containing temperature changes
df_temp = df[[col for col in df.columns if 'temperature' in col]]

#Columns for the methane and NOx
df_chem = df[['year','methane','nitrous_oxide']]

<h3>Table 1: Sources Table</h3>

In [508]:
# By pd.melt, reverse the columns into rows and maintain the values
df_sources =pd.melt(df_sources.reset_index(),id_vars=['index'],value_vars=df_sources.columns, \
    var_name="emission_sources",value_name="million_tons").copy()
    
df_sources = df_sources.groupby('emission_sources')['million_tons'].sum().reset_index()

In [509]:
# Adding the Percentage Column
df_sources['percentage'] = (df_sources['million_tons']/df_sources['million_tons'].sum())*100
df_sources['percentage'] = df_sources['percentage'].apply(lambda x: round(x,0) if x > 1 else round(x,2))
df_sources

,emission_sources,million_tons,percentage
0,cement_co2,209594.066,1.00
1,coal_co2,3870596.153,21.00
2,consumption_co2,6073187.268,33.00
3,flaring_co2,86909.154,0.47
4,gas_co2,1249162.183,7.00
5,land_use_change_co2,4333960.833,23.00
6,oil_co2,2789318.374,15.00
7,other_industry_co2,43974.095,0.24


<h3> Grouping By Country</h3>
<p>based on checking the dataset structure, the rows represents by the categorical country per parameters of CO<sub>2</sub>. I will take the mean value of the emissions to compress the dataset, and simplify its findings. </p>

In [510]:
# Set country as index
df_demographics.set_index(["country"], inplace=True)

# Column with values to maintain
demographics_values = ["population","gdp","co2","land_use_change_co2"]
df_demo_country = df_demographics.groupby("country") \
    [demographics_values].mean()
df_demo_country.reset_index(inplace=True)

df_demo_country.shape

(232, 5)

In [511]:
# Rename co2 column based on the data definition
df_demo_country = df_demo_country.rename(columns={"co2":"co2 without land_use", \
    "index":"id"})

# Include land use change for calculations
df_demo_country['co2'] = df_demo_country['co2 without land_use'] + \
    demographics_df['land_use_change_co2']

# Dropping unnecessary columns
df_demo_country = df_demo_country.drop(['land_use_change_co2'],axis=1).copy()
df_demo_country.head()

,country,population,gdp,co2 without land_use,co2
0,Afghanistan,1.018891e+07,2.146437e+10,2.986219,5.917219
1,Aland Islands,NaN,NaN,NaN,NaN
2,Albania,1.573110e+06,1.160662e+10,3.301562,6.269562
3,Algeria,1.276109e+07,1.651044e+11,45.873245,48.877245
4,American Samoa,3.527511e+04,NaN,NaN,NaN


In [512]:
df_demo_country.shape

(232, 5)

<h3> Grouping By Year</h3>
<p>based on checking the dataset structure, the rows represents by year per parameters of CO<sub>2</sub>. I will take the mean value of the emissions to compress the dataset, and simplify its findings. </p>

In [513]:
# Set year as index
df_demographics.set_index(["year"], inplace=True)

# Column with values to maintain
demographics_values = ["population","gdp","co2","land_use_change_co2"]
df_demo_yr = df_demographics.groupby("year") \
    [demographics_values].mean()
df_demo_yr.reset_index(inplace=True)

df_demo_yr.shape

(272, 5)

In [514]:
# Rename co2 column based on the data definition
df_demo_yr = df_demo_yr.rename(columns={"co2":"co2 without land_use", \
    "index":"id"})

# Include land use change for calculations
df_demo_yr['co2'] = df_demo_yr['co2 without land_use'] + \
    demographics_df['land_use_change_co2']

# Dropping unnecessary columns
df_demo_yr = df_demo_yr.drop(['land_use_change_co2'],axis=1).copy()
df_demo_yr.head()

,year,population,gdp,co2 without land_use,co2
0,1750,939415.764706,NaN,0.519500,3.450500
1,1751,NaN,NaN,0.519500,3.487500
2,1752,NaN,NaN,0.519667,3.487667
3,1753,NaN,NaN,0.519667,3.523667
4,1754,NaN,NaN,0.519889,3.523889


<h3> Temperature Change and CO<sub>2</sub> Emission </h3>


In [517]:
df_temp.columns
# df_temp
# df_chem

Index(['share_of_temperature_change_from_ghg', 'temperature_change_from_ch4',
       'temperature_change_from_co2', 'temperature_change_from_ghg',
       'temperature_change_from_n2o'],
      dtype='object')

<h3>Adding Foreign Key to Other Tables</h3>

In [ ]:

# Add Chemical Sources Key
filtered_df['chem_key'] = [int("20" + str(0 + x)) for x in range(len(filtered_df))]


# Add Temperature Change Key
filtered_df['temp_key'] = [int("30" + str(0+x)) for x in range(len(filtered_df))]


Key_columns = [col for col in filtered_df.columns if 'key' in col]
Key_columns


['country_key', 'chem_key', 'temp_key']

In [ ]:
#Loop to drop and insert the key columns to the left side of the dataframe
for key in Key_columns:
    filtered_df[key] = filtered_df[key].astype(str)
    foreign_keys = filtered_df.pop(key)  
    filtered_df.insert(0, key, foreign_keys)

In [ ]:
filtered_df.head()

,temp_key,chem_key,country_key,country,year,iso_code,population,gdp,co2,land_use_change_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,year,methane,nitrous_oxide
0,300,200,100,Afghanistan,1850,AFG,3752993.0,NaN,NaN,2.931,NaN,NaN,NaN,NaN,NaN,1850,NaN,NaN
1,301,201,100,Afghanistan,1851,AFG,3767956.0,NaN,NaN,2.968,0.165,0.0,0.0,0.0,0.0,1851,NaN,NaN
2,302,202,100,Afghanistan,1852,AFG,3783940.0,NaN,NaN,2.968,0.164,0.0,0.0,0.0,0.0,1852,NaN,NaN
3,303,203,100,Afghanistan,1853,AFG,3800954.0,NaN,NaN,3.004,0.164,0.0,0.0,0.0,0.0,1853,NaN,NaN
4,304,204,100,Afghanistan,1854,AFG,3818038.0,NaN,NaN,3.004,0.163,0.0,0.0,0.0,0.0,1854,NaN,NaN


<p>Summarizing Country Table, Chemical Composition Table, and Temperature Change Table</p>

In [ ]:
country_df = filtered_df[['country_key','country','iso_code']]
chem_df = filtered_df[['chem_key','methane','nitrous_oxide']]
temp_df = filtered_df[['temp_key','share_of_temperature_change_from_ghg',\
    'temperature_change_from_ch4','temperature_change_from_co2','temperature_change_from_ghg', \
    'temperature_change_from_n2o']]


In [ ]:
country_df = country_df.drop_duplicates(subset=['country']).copy()
country_df.dropna(subset=['iso_code'],inplace=True)
country_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 232 entries, 0 to 50426
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   country_key  232 non-null    object
 1   country      232 non-null    object
 2   iso_code     232 non-null    object
dtypes: object(3)
memory usage: 7.2+ KB


In [ ]:
chem_df = chem_df.dropna(subset=['methane','nitrous_oxide']).copy()
chem_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6150 entries, 140 to 50595
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   chem_key       6150 non-null   object 
 1   methane        6150 non-null   float64
 2   nitrous_oxide  6150 non-null   float64
dtypes: float64(2), object(1)
memory usage: 192.2+ KB


In [ ]:
temp_df = temp_df.dropna(subset=['temperature_change_from_ch4',\
    'temperature_change_from_n2o']).copy()
    
temp_df['share_of_temperature_change_from_ghg']. \
    fillna(df['share_of_temperature_change_from_ghg'].mean())

temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37620 entries, 1 to 50597
Data columns (total 6 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   temp_key                              37620 non-null  object 
 1   share_of_temperature_change_from_ghg  37620 non-null  float64
 2   temperature_change_from_ch4           37620 non-null  float64
 3   temperature_change_from_co2           37620 non-null  float64
 4   temperature_change_from_ghg           37620 non-null  float64
 5   temperature_change_from_n2o           37620 non-null  float64
dtypes: float64(5), object(1)
memory usage: 2.0+ MB


<p>Demographics Table</p>

In [ ]:
demographics_df = filtered_df.drop(['share_of_temperature_change_from_ghg', \
    'temperature_change_from_ch4','temperature_change_from_co2', 'temperature_change_from_ghg', \
    'temperature_change_from_n2o', 'methane', 'nitrous_oxide'],axis=1)

demographics_df.drop_duplicates(subset=['year'],inplace=True)

In [ ]:
grouped_demo = demographics_df.groupby('country').mean()
grouped_demo

TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
# Drop NaN values from CO2 emission columns
# demographics_df = demographics_df.dropna(subset=['co2','land_use_change_co2']).copy()

# Rename co2 column based on the data definition
demographics_df = demographics_df.rename(columns={"co2":"co2 without land_use"}).reset_index()

# Include land use change for calculations
demographics_df['co2'] = demographics_df['co2 without land_use'] + demographics_df['land_use_change_co2']

# Dropping unnecessary columns
demographics_df = demographics_df.drop(['land_use_change_co2'],axis=1).copy()



In [ ]:
print(demographics_df)

     index temp_key chem_key country_key      country  year iso_code  \
0        0      300      200         100  Afghanistan  1850      AFG   
1        1      301      201         100  Afghanistan  1851      AFG   
2        2      302      202         100  Afghanistan  1852      AFG   
3        3      303      203         100  Afghanistan  1853      AFG   
4        4      304      204         100  Afghanistan  1854      AFG   
..     ...      ...      ...         ...          ...   ...      ...   
267    267    30267    20267         101       Africa  1845      NaN   
268    268    30268    20268         101       Africa  1846      NaN   
269    269    30269    20269         101       Africa  1847      NaN   
270    270    30270    20270         101       Africa  1848      NaN   
271    271    30271    20271         101       Africa  1849      NaN   

      population  gdp  co2 without land_use  year  co2  
0      3752993.0  NaN                   NaN  1850  NaN  
1      3767956.0  NaN

In [ ]:
demographics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272 entries, 0 to 271
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 272 non-null    int64  
 1   temp_key              272 non-null    object 
 2   chem_key              272 non-null    object 
 3   country_key           272 non-null    object 
 4   country               272 non-null    object 
 5   year                  272 non-null    int64  
 6   iso_code              172 non-null    object 
 7   population            227 non-null    float64
 8   gdp                   69 non-null     float64
 9   co2 without land_use  173 non-null    float64
 10  year                  272 non-null    int64  
 11  co2                   73 non-null     float64
dtypes: float64(4), int64(3), object(5)
memory usage: 25.6+ KB


<h3>Saving the processed dataset</h3>

In [ ]:
# #Sources Table
# df_sources.to_csv("clean_data/emission_sources.csv",index=False)

# # Country Table
# country_df.to_csv("clean_data/country.csv",index=False)

# #  Chemical Composition Table
# chem_df.to_csv("clean_data/chemical.csv",index=False)

# # Temperature Change Table
# temp_df.to_csv("clean_data/temperature_change.csv",index=False)

# # Demographics Table
# demographics_df.to_csv("clean_data/demographics.csv",index=False)